# Phase 5: Depth (Y-axis) + Dual-Axis Rotation Alignment

## Center-Based Approach

### Новий алгоритм (підхід професора):

1. **Overlap Region Detection**: Знайти спільну область після XZ alignment
2. **Center-based Y Alignment**: Центр overlap → зсув щоб центри співпали
3. **X-axis Rotation** (±10°): Tilt forward/backward навколо центру
4. **Z-axis Rotation** (±30°): Rotation in YX plane навколо центру
5. **Apply to Full Volumes**: Застосувати всі трансформації

### Переваги:
- ✅ Природна точка відліку (центр overlap)
- ✅ Обертання зберігає максимальний overlap
- ✅ Порівняння тільки реальних даних
- ✅ Більш стабільні результати

### Метрика: NCC (Normalized Cross-Correlation)
- Стабільна для medical imaging
- Нормалізована до яскравості
- Швидка та надійна

## 1. Setup and Imports

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import ndimage
from scipy.ndimage import rotate
import time

# Add src to path - robust version
notebook_dir = Path.cwd()
if notebook_dir.name == 'notebooks':
    src_dir = notebook_dir.parent / 'src'
    data_dir = notebook_dir / 'data'
    oct_data_dir = notebook_dir.parent / 'oct_data'
else:
    # Running from root or other location
    src_dir = Path('/home/illia/diploma/RetinaBuilder/src')
    data_dir = Path('/home/illia/diploma/RetinaBuilder/notebooks/data')
    oct_data_dir = Path('/home/illia/diploma/RetinaBuilder/oct_data')

sys.path.insert(0, str(src_dir))

# Now import
from oct_volumetric_viewer import OCTImageProcessor, OCTVolumeLoader

plt.rcParams['figure.figsize'] = (15, 10)

print("✓ Imports complete")
print(f"📁 Src directory: {src_dir}")
print(f"📁 Data directory: {data_dir}")
print(f"📁 OCT data directory: {oct_data_dir}")

## 2. Load X-Z Registration Results

In [ ]:
# Load X-Z registration parameters
xy_params = np.load(data_dir / 'xy_registration_params.npy', allow_pickle=True).item()

print("="*70)
print("📊 X-Z Registration Parameters (from Phase 3)")
print("="*70)
print(f"  Method: {xy_params.get('best_method', 'phase_correlation')}")
print(f"  X offset: {xy_params['offset_x']} pixels")
print(f"  Z offset: {xy_params['offset_z']} pixels")
print(f"  Confidence: {xy_params['confidence']:.2f}")
print(f"  Improvement: {xy_params['improvement_percent']:.1f}%")
print("="*70)

offset_x = xy_params['offset_x']
offset_z = xy_params['offset_z']

## 3. Load Full 3D Volumes

In [ ]:
print("Loading full 3D OCT volumes...\n")

start_load = time.time()

processor = OCTImageProcessor(sidebar_width=250, crop_top=100, crop_bottom=50)
loader = OCTVolumeLoader(processor)

# Find F001 volumes
bmp_dirs = []
for bmp_file in oct_data_dir.rglob('*.bmp'):
    vol_dir = bmp_file.parent
    if vol_dir not in bmp_dirs:
        bmp_dirs.append(vol_dir)

all_volume_dirs = sorted(bmp_dirs)
f001_vols = [v for v in all_volume_dirs if 'F001_IP' in str(v)]

if len(f001_vols) >= 2:
    volume_dirs = f001_vols[:2]
else:
    volume_dirs = all_volume_dirs[:2]

# Load
volume_0 = loader.load_volume_from_directory(str(volume_dirs[0]))
volume_1 = loader.load_volume_from_directory(str(volume_dirs[1]))

print(f"✓ Loaded volumes in {time.time()-start_load:.1f}s")
print(f"  Volume 0: {volume_0.shape} (Y, X, Z)")
print(f"  Volume 1: {volume_1.shape}")

## 4. Apply X-Z Translation

In [ ]:
volume_1_xz_aligned = ndimage.shift(
    volume_1, shift=(0, offset_x, offset_z),
    order=1, mode='constant', cval=0
)
print(f"✓ Applied XZ alignment: X={offset_x}, Z={offset_z}")

## 5. Calculate Overlap Region

Знаходимо спільну область в XZ площині.

In [ ]:
def calculate_overlap_region(v0_shape, offset_x, offset_z):
    Y, X, Z = v0_shape
    
    # X overlap
    if offset_x >= 0:
        x0_start, x0_end = offset_x, X
        x1_start, x1_end = 0, X - offset_x
    else:
        x0_start, x0_end = 0, X + offset_x
        x1_start, x1_end = -offset_x, X
    
    # Z overlap
    if offset_z >= 0:
        z0_start, z0_end = offset_z, Z
        z1_start, z1_end = 0, Z - offset_z
    else:
        z0_start, z0_end = 0, Z + offset_z
        z1_start, z1_end = -offset_z, Z
    
    return {
        'v0': {'x': (x0_start, x0_end), 'z': (z0_start, z0_end)},
        'v1': {'x': (x1_start, x1_end), 'z': (z1_start, z1_end)},
        'size': (Y, x0_end-x0_start, z0_end-z0_start)
    }

overlap = calculate_overlap_region(volume_0.shape, offset_x, offset_z)
print("="*70)
print("OVERLAP REGION")
print("="*70)
print(f"  Size: {overlap['size']}")
print(f"  V0: X[{overlap['v0']['x'][0]}:{overlap['v0']['x'][1]}], Z[{overlap['v0']['z'][0]}:{overlap['v0']['z'][1]}]")
print(f"  V1: X[{overlap['v1']['x'][0]}:{overlap['v1']['x'][1]}], Z[{overlap['v1']['z'][0]}:{overlap['v1']['z'][1]}]")
print("="*70)

## 6. Extract Overlap Regions

In [ ]:
# Extract overlap regions
x0 = overlap['v0']['x']
z0 = overlap['v0']['z']
x1 = overlap['v1']['x']
z1 = overlap['v1']['z']

overlap_v0 = volume_0[:, x0[0]:x0[1], z0[0]:z0[1]].copy()
overlap_v1 = volume_1_xz_aligned[:, x1[0]:x1[1], z1[0]:z1[1]].copy()

print(f"✓ Overlap V0: {overlap_v0.shape}")
print(f"✓ Overlap V1: {overlap_v1.shape}")

## 7. Center-based Y Alignment

Знаходимо центри overlap region по Y осі.

In [ ]:
def find_y_center(volume):
    """Центр маси по Y осі."""
    y_profile = volume.sum(axis=(1, 2))
    y_coords = np.arange(len(y_profile))
    center = np.average(y_coords, weights=y_profile + 1e-8)
    return center

center_y_v0 = find_y_center(overlap_v0)
center_y_v1 = find_y_center(overlap_v1)
y_shift = center_y_v0 - center_y_v1

print("="*70)
print("Y-AXIS CENTER POINTS")
print("="*70)
print(f"  V0 center Y: {center_y_v0:.2f}")
print(f"  V1 center Y: {center_y_v1:.2f}")
print(f"  Y shift needed: {y_shift:+.2f} px")
print("="*70)

# Apply Y shift to overlap V1
overlap_v1_y_aligned = ndimage.shift(
    overlap_v1, shift=(y_shift, 0, 0),
    order=1, mode='constant', cval=0
)
print(f"\n✓ Applied Y shift: {y_shift:+.2f} px")
print(f"  New center Y: {find_y_center(overlap_v1_y_aligned):.2f}")

## 8. NCC Metric Function

Normalized Cross-Correlation для порівняння зображень.

In [ ]:
def calculate_ncc(img1, img2, mask=None):
    """
    Normalized Cross-Correlation.
    
    NCC = mean((img1_norm * img2_norm))
    Range: -1 to 1 (1 = perfect match)
    
    Args:
        img1, img2: Images to compare
        mask: Optional boolean mask - compare only True pixels
    """
    if mask is not None:
        # Use only valid pixels
        img1 = img1[mask]
        img2 = img2[mask]
    
    if len(img1) == 0 or len(img2) == 0:
        return -1.0
    
    img1_norm = (img1 - img1.mean()) / (img1.std() + 1e-8)
    img2_norm = (img2 - img2.mean()) / (img2.std() + 1e-8)
    ncc = np.mean(img1_norm * img2_norm)
    return float(ncc)

# Test on current alignment - ВАЖЛИВО: з tissue threshold!
threshold_before = calculate_tissue_threshold(overlap_v0, overlap_v1_y_aligned, percentile=50)
mask_before_rotation = (overlap_v0 > threshold_before) & (overlap_v1_y_aligned > threshold_before)
ncc_before_rotation = calculate_ncc(overlap_v0, overlap_v1_y_aligned, mask=mask_before_rotation)

print(f"NCC after Y alignment (before rotation): {ncc_before_rotation:.4f}")
print(f"Tissue threshold: {threshold_before:.1f}")
print(f"Valid tissue pixels: {mask_before_rotation.sum():,} / {overlap_v0.size:,} ({100*mask_before_rotation.sum()/overlap_v0.size:.1f}%)")

In [ ]:
def calculate_tissue_threshold(img1, img2, percentile=50):
    """
    Calculate threshold based on tissue intensity distribution.
    
    Uses 50th percentile (median) of non-zero pixels to filter
    only bright retinal structures, ignoring noise and weak signals.
    
    Args:
        img1, img2: Images to analyze
        percentile: Percentile of non-zero pixels (default 50 = median)
    
    Returns:
        threshold: Average threshold for both images
    """
    nz1 = img1[img1 > 0]
    nz2 = img2[img2 > 0]
    
    thresh1 = np.percentile(nz1, percentile) if len(nz1) > 0 else 0
    thresh2 = np.percentile(nz2, percentile) if len(nz2) > 0 else 0
    
    threshold = (thresh1 + thresh2) / 2
    
    return threshold

def find_valid_overlap_bounds(ref_slice, mov_slice, threshold):
    """
    Find bounding box where BOTH slices have tissue above threshold.
    
    This ensures we compare only the region where both volumes
    have actual tissue data, not cropped/empty regions.
    
    Args:
        ref_slice, mov_slice: 2D slices to compare
        threshold: Intensity threshold for tissue
    
    Returns:
        (y_min, y_max, x_min, x_max) or None if insufficient overlap
    """
    mask_ref = ref_slice > threshold
    mask_mov = mov_slice > threshold
    mask_both = mask_ref & mask_mov
    
    if mask_both.sum() < 100:
        return None  # No sufficient overlap
    
    # Find bounding box
    coords = np.argwhere(mask_both)
    y_min, x_min = coords.min(axis=0)
    y_max, x_max = coords.max(axis=0)
    
    # Add small padding
    padding = 5
    y_min = max(0, y_min - padding)
    y_max = min(ref_slice.shape[0], y_max + padding + 1)
    x_min = max(0, x_min - padding)
    x_max = min(ref_slice.shape[1], x_max + padding + 1)
    
    return (y_min, y_max, x_min, x_max)

print("✓ Tissue threshold functions defined")

In [ ]:
def find_best_x_rotation(overlap_ref, overlap_mov, center_y, angle_range=10, step=2):
    """
    X-axis rotation: tilt в площині YZ.
    axes=(0, 2) = (Y, Z) rotation
    
    OPTIMIZED: використовує центральний B-scan замість всього volume.
    IMPORTANT: порівнює ТІЛЬКИ valid tissue overlap з robust threshold.
    """
    angles = range(-angle_range, angle_range + 1, step)
    results = []
    best_angle = 0
    best_ncc = -1
    
    # Get center B-scan
    x_center = overlap_ref.shape[1] // 2
    ref_slice = overlap_ref[:, x_center, :].copy()  # Shape: (Y, Z)
    mov_slice = overlap_mov[:, x_center, :].copy()
    
    # Calculate tissue threshold (50th percentile)
    threshold = calculate_tissue_threshold(ref_slice, mov_slice, percentile=50)
    
    # Find valid overlap bounds
    bounds = find_valid_overlap_bounds(ref_slice, mov_slice, threshold)
    
    if bounds is None:
        print("⚠️ ERROR: No sufficient tissue overlap for X-axis rotation!")
        return 0, -1, []
    
    # Crop to valid overlap region
    y_min, y_max, z_min, z_max = bounds
    ref_crop = ref_slice[y_min:y_max, z_min:z_max].copy()
    mov_crop = mov_slice[y_min:y_max, z_min:z_max].copy()
    
    print(f"Testing {len(angles)} X-axis angles...")
    print(f"  Using center B-scan X={x_center}")
    print(f"  Tissue threshold: {threshold:.1f}")
    print(f"  Cropped to: Y[{y_min}:{y_max}] x Z[{z_min}:{z_max}] = {ref_crop.shape}")
    
    for angle in angles:
        # Rotate cropped slice (Y, Z)
        rotated = rotate(
            mov_crop, angle, axes=(0, 1),  # 2D rotation
            reshape=False, order=1, mode='constant', cval=0
        )
        
        # ВАЖЛИВО: mask з tissue threshold (не просто >0!)
        mask = (ref_crop > threshold) & (rotated > threshold)
        
        # NCC тільки на tissue pixels
        if mask.sum() > 100:  # Потрібно мінімум 100 пікселів
            ncc = calculate_ncc(ref_crop, rotated, mask=mask)
        else:
            ncc = -1  # Недостатньо tissue overlap
        
        results.append({'angle': angle, 'ncc': ncc, 'valid_pixels': int(mask.sum())})
        
        if ncc > best_ncc:
            best_ncc = ncc
            best_angle = angle
    
    print(f"✓ Best X-axis angle: {best_angle}° (NCC={best_ncc:.4f})")
    return best_angle, best_ncc, results

print("="*70)
print("X-AXIS ROTATION SEARCH")
print("="*70)
start_x = time.time()

best_x_angle, best_x_ncc, x_results = find_best_x_rotation(
    overlap_v0, overlap_v1_y_aligned, center_y_v0,
    angle_range=10, step=2
)

print(f"  Time: {time.time()-start_x:.1f}s")
print("="*70)

## 8c. DIAGNOSTIC: Visualize Slices Before Rotation

**ВАЖЛИВО:** Перед rotation треба побачити які зрізи ми порівнюємо!

# Apply best X rotation to overlap V1
overlap_v1_x_rotated = rotate(
    overlap_v1_y_aligned, best_x_angle, axes=(0, 2),
    reshape=False, order=1, mode='constant', cval=0
)

# ВАЖЛИВО: NCC з tissue threshold
threshold_after_x = calculate_tissue_threshold(overlap_v0, overlap_v1_x_rotated, percentile=50)
mask_after_x = (overlap_v0 > threshold_after_x) & (overlap_v1_x_rotated > threshold_after_x)
ncc_after_x = calculate_ncc(overlap_v0, overlap_v1_x_rotated, mask=mask_after_x)

print(f"✓ Applied X rotation: {best_x_angle}°")
print(f"  Tissue threshold: {threshold_after_x:.1f}")
print(f"  Valid tissue pixels: {mask_after_x.sum():,} / {overlap_v0.size:,} ({100*mask_after_x.sum()/overlap_v0.size:.1f}%)")
print(f"  NCC improved: {ncc_before_rotation:.4f} → {ncc_after_x:.4f}")
print(f"  Δ NCC: {ncc_after_x - ncc_before_rotation:+.4f}")

## 9. X-axis Rotation Search (±10°)

Tilt forward/backward навколо центральної точки.
Обертання в площині YZ (навколо X-осі).

In [ ]:
def find_best_z_rotation(overlap_ref, overlap_mov, angle_range=30, step=2):
    """
    Z-axis rotation: rotation в площині YX.
    axes=(0, 1) = (Y, X) rotation
    
    OPTIMIZED: використовує центральний Y-slice замість всього volume.
    IMPORTANT: порівнює ТІЛЬКИ valid tissue overlap з robust threshold.
    """
    angles = range(-angle_range, angle_range + 1, step)
    results = []
    best_angle = 0
    best_ncc = -1
    
    # Get center Y-slice
    y_center = overlap_ref.shape[0] // 2
    ref_slice = overlap_ref[y_center, :, :].copy()  # Shape: (X, Z)
    mov_slice = overlap_mov[y_center, :, :].copy()
    
    # Calculate tissue threshold (50th percentile)
    threshold = calculate_tissue_threshold(ref_slice, mov_slice, percentile=50)
    
    # Find valid overlap bounds
    bounds = find_valid_overlap_bounds(ref_slice, mov_slice, threshold)
    
    if bounds is None:
        print("⚠️ ERROR: No sufficient tissue overlap for Z-axis rotation!")
        return 0, -1, []
    
    # Crop to valid overlap region
    x_min, x_max, z_min, z_max = bounds
    ref_crop = ref_slice[x_min:x_max, z_min:z_max].copy()
    mov_crop = mov_slice[x_min:x_max, z_min:z_max].copy()
    
    print(f"Testing {len(angles)} Z-axis angles...")
    print(f"  Using center Y-slice Y={y_center}")
    print(f"  Tissue threshold: {threshold:.1f}")
    print(f"  Cropped to: X[{x_min}:{x_max}] x Z[{z_min}:{z_max}] = {ref_crop.shape}")
    
    for angle in angles:
        # Rotate cropped slice (X, Z)
        rotated = rotate(
            mov_crop, angle, axes=(0, 1),  # 2D rotation
            reshape=False, order=1, mode='constant', cval=0
        )
        
        # ВАЖЛИВО: mask з tissue threshold (не просто >0!)
        mask = (ref_crop > threshold) & (rotated > threshold)
        
        # NCC тільки на tissue pixels
        if mask.sum() > 100:  # Потрібно мінімум 100 пікселів
            ncc = calculate_ncc(ref_crop, rotated, mask=mask)
        else:
            ncc = -1  # Недостатньо tissue overlap
        
        results.append({'angle': angle, 'ncc': ncc, 'valid_pixels': int(mask.sum())})
        
        if ncc > best_ncc:
            best_ncc = ncc
            best_angle = angle
    
    print(f"✓ Best Z-axis angle: {best_angle}° (NCC={best_ncc:.4f})")
    return best_angle, best_ncc, results

print("="*70)
print("Z-AXIS ROTATION SEARCH")
print("="*70)
start_z = time.time()

best_z_angle, best_z_ncc, z_results = find_best_z_rotation(
    overlap_v0, overlap_v1_x_rotated,
    angle_range=30, step=2
)

print(f"  Time: {time.time()-start_z:.1f}s")
print("="*70)

## 10. Apply X-axis Rotation

In [ ]:
# Apply best Z rotation
overlap_v1_fully_aligned = rotate(
    overlap_v1_x_rotated, best_z_angle, axes=(0, 1),
    reshape=False, order=1, mode='constant', cval=0
)

# ВАЖЛИВО: NCC з tissue threshold
threshold_final = calculate_tissue_threshold(overlap_v0, overlap_v1_fully_aligned, percentile=50)
mask_final = (overlap_v0 > threshold_final) & (overlap_v1_fully_aligned > threshold_final)
ncc_final = calculate_ncc(overlap_v0, overlap_v1_fully_aligned, mask=mask_final)

print(f"✓ Applied Z rotation: {best_z_angle}°")
print(f"  Tissue threshold: {threshold_final:.1f}")
print(f"  Valid tissue pixels: {mask_final.sum():,} / {overlap_v0.size:,} ({100*mask_final.sum()/overlap_v0.size:.1f}%)")

print(f"\n📊 NCC Progress:")
print(f"  After Y align: {ncc_before_rotation:.4f}")
print(f"  After X rot:   {ncc_after_x:.4f} ({ncc_after_x-ncc_before_rotation:+.4f})")
print(f"  After Z rot:   {ncc_final:.4f} ({ncc_final-ncc_after_x:+.4f})")
print(f"  Total gain:    {ncc_final-ncc_before_rotation:+.4f}")

## 11. Z-axis Rotation Search (±30°)

Rotation in YX plane (як робили раніше).
Обертання навколо Z-осі.

In [ ]:
def find_best_z_rotation(overlap_ref, overlap_mov, angle_range=30, step=2):
    """
    Z-axis rotation: rotation в площині YX.
    axes=(0, 1) = (Y, X) rotation
    
    OPTIMIZED: використовує центральний Y-slice замість всього volume.
    IMPORTANT: порівнює ТІЛЬКИ ті пікселі що залишаються спільними після rotation.
    """
    angles = range(-angle_range, angle_range + 1, step)
    results = []
    best_angle = 0
    best_ncc = -1
    
    # ОПТИМІЗАЦІЯ: використовуємо ОДИН центральний Y-slice
    y_center = overlap_ref.shape[0] // 2
    ref_slice = overlap_ref[y_center, :, :].copy()  # Shape: (X, Z)
    mov_slice = overlap_mov[y_center, :, :].copy()
    
    print(f"Testing {len(angles)} Z-axis angles (using center Y-slice Y={y_center})...")
    
    for angle in angles:
        # Rotate 2D slice (X, Z)
        rotated = rotate(
            mov_slice, angle, axes=(0, 1),  # 2D rotation
            reshape=False, order=1, mode='constant', cval=0
        )
        
        # ВАЖЛИВО: mask для valid pixels (де обидва не нульові)
        mask = (ref_slice > 0) & (rotated > 0)
        
        # NCC тільки на спільних пікселях
        if mask.sum() > 100:  # Потрібно мінімум 100 пікселів
            ncc = calculate_ncc(ref_slice, rotated, mask=mask)
        else:
            ncc = -1  # Недостатньо overlap
        
        results.append({'angle': angle, 'ncc': ncc, 'valid_pixels': int(mask.sum())})
        
        if ncc > best_ncc:
            best_ncc = ncc
            best_angle = angle
    
    print(f"✓ Best Z-axis angle: {best_angle}° (NCC={best_ncc:.4f})")
    return best_angle, best_ncc, results

print("="*70)
print("Z-AXIS ROTATION SEARCH")
print("="*70)
start_z = time.time()

best_z_angle, best_z_ncc, z_results = find_best_z_rotation(
    overlap_v0, overlap_v1_x_rotated,
    angle_range=30, step=2
)

print(f"  Time: {time.time()-start_z:.1f}s")
print("="*70)

## 12. Apply Z-axis Rotation

In [ ]:
# Apply best Z rotation
overlap_v1_fully_aligned = rotate(
    overlap_v1_x_rotated, best_z_angle, axes=(0, 1),
    reshape=False, order=1, mode='constant', cval=0
)

# ВАЖЛИВО: NCC з mask для valid pixels
mask_final = (overlap_v0 > 0) & (overlap_v1_fully_aligned > 0)
ncc_final = calculate_ncc(overlap_v0, overlap_v1_fully_aligned, mask=mask_final)

print(f"✓ Applied Z rotation: {best_z_angle}°")
print(f"  Valid pixels: {mask_final.sum():,} / {overlap_v0.size:,} ({100*mask_final.sum()/overlap_v0.size:.1f}%)")

print(f"\n📊 NCC Progress:")
print(f"  After Y align: {ncc_before_rotation:.4f}")
print(f"  After X rot:   {ncc_after_x:.4f} ({ncc_after_x-ncc_before_rotation:+.4f})")
print(f"  After Z rot:   {ncc_final:.4f} ({ncc_final-ncc_after_x:+.4f})")
print(f"  Total gain:    {ncc_final-ncc_before_rotation:+.4f}")

## 13. Apply Transformations to Full Volumes

Застосувати всі знайдені трансформації до повних volumes.

In [ ]:
print("Applying transformations to full Volume 1...\n")
start_apply = time.time()

# Start with XZ-aligned volume
v1_transformed = volume_1_xz_aligned.copy()

# Step 1: Y shift
print(f"Step 1: Y shift {y_shift:+.2f} px")
v1_transformed = ndimage.shift(
    v1_transformed, shift=(y_shift, 0, 0),
    order=1, mode='constant', cval=0
)

# Step 2: X-axis rotation
print(f"Step 2: X-axis rotation {best_x_angle}°")
v1_transformed = rotate(
    v1_transformed, best_x_angle, axes=(0, 2),
    reshape=False, order=1, mode='constant', cval=0
)

# Step 3: Z-axis rotation
print(f"Step 3: Z-axis rotation {best_z_angle}°")
v1_transformed = rotate(
    v1_transformed, best_z_angle, axes=(0, 1),
    reshape=False, order=1, mode='constant', cval=0
)

print(f"\n✓ Transformation applied in {time.time()-start_apply:.1f}s")
print(f"  Final volume shape: {v1_transformed.shape}")

## 14. Calculate Alignment Quality

In [ ]:
# Compare before/after on full volumes
diff_before = np.abs(volume_0.astype(float) - volume_1_xz_aligned.astype(float))
diff_after = np.abs(volume_0.astype(float) - v1_transformed.astype(float))

improvement = 100 * (1 - diff_after.mean() / diff_before.mean())

print("="*70)
print("ALIGNMENT QUALITY")
print("="*70)
print(f"  Before (XZ only): {diff_before.mean():.2f} ± {diff_before.std():.2f}")
print(f"  After (XYZ + rot): {diff_after.mean():.2f} ± {diff_after.std():.2f}")
print(f"\n  Improvement: {improvement:.2f}%")

if improvement > 10:
    print("  ✅ EXCELLENT alignment!")
elif improvement > 5:
    print("  ✅ GOOD alignment!")
elif improvement > 0:
    print("  ⚠️ MODERATE improvement")
else:
    print("  ❌ No improvement - check parameters")
print("="*70)

## 15. Visualization

In [ ]:
# Visualize rotation search results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# X-axis rotation curve
x_angles = [r['angle'] for r in x_results]
x_nccs = [r['ncc'] for r in x_results]
axes[0, 0].plot(x_angles, x_nccs, 'b-o', linewidth=2)
axes[0, 0].axvline(best_x_angle, color='red', linestyle='--', label=f'Best: {best_x_angle}°')
axes[0, 0].set_xlabel('X-axis Angle (degrees)')
axes[0, 0].set_ylabel('NCC Score')
axes[0, 0].set_title('X-axis Rotation Search')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].legend()

# Z-axis rotation curve
z_angles = [r['angle'] for r in z_results]
z_nccs = [r['ncc'] for r in z_results]
axes[0, 1].plot(z_angles, z_nccs, 'g-o', linewidth=2)
axes[0, 1].axvline(best_z_angle, color='red', linestyle='--', label=f'Best: {best_z_angle}°')
axes[0, 1].set_xlabel('Z-axis Angle (degrees)')
axes[0, 1].set_ylabel('NCC Score')
axes[0, 1].set_title('Z-axis Rotation Search')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].legend()

# B-scan comparison (center)
z_mid = volume_0.shape[2] // 2
axes[1, 0].imshow(volume_0[:, :, z_mid], cmap='Reds', alpha=0.5, aspect='auto')
axes[1, 0].imshow(volume_1_xz_aligned[:, :, z_mid], cmap='Greens', alpha=0.5, aspect='auto')
axes[1, 0].set_title(f'Before (XZ only)\nZ={z_mid}')
axes[1, 0].set_ylabel('Y (depth)')
axes[1, 0].set_xlabel('X (lateral)')

axes[1, 1].imshow(volume_0[:, :, z_mid], cmap='Reds', alpha=0.5, aspect='auto')
axes[1, 1].imshow(v1_transformed[:, :, z_mid], cmap='Greens', alpha=0.5, aspect='auto')
axes[1, 1].set_title(f'After (XYZ + Rot)\nImprovement: {improvement:.1f}%')
axes[1, 1].set_ylabel('Y (depth)')
axes[1, 1].set_xlabel('X (lateral)')

plt.tight_layout()
plt.show()

## 16. Save Results

In [ ]:
# Save aligned volume
np.save(data_dir / 'volume_1_fully_aligned.npy', v1_transformed)
print("✓ Saved: volume_1_fully_aligned.npy")

# Save parameters
depth_alignment_params = {
    'method': 'center_based_dual_axis_rotation',
    'y_shift': float(y_shift),
    'center_y_v0': float(center_y_v0),
    'center_y_v1': float(center_y_v1),
    'x_rotation_angle': float(best_x_angle),
    'x_rotation_ncc': float(best_x_ncc),
    'z_rotation_angle': float(best_z_angle),
    'z_rotation_ncc': float(best_z_ncc),
    'ncc_before': float(ncc_before_rotation),
    'ncc_after': float(ncc_final),
    'improvement_percent': float(improvement),
    'overlap_size': overlap['size']
}
np.save(data_dir / 'depth_alignment_params.npy', depth_alignment_params, allow_pickle=True)
print("✓ Saved: depth_alignment_params.npy")

# Complete 3D registration
registration_3d = {
    'xz_alignment': {
        'x_offset': int(offset_x),
        'z_offset': int(offset_z),
        'method': xy_params.get('best_method'),
        'confidence': float(xy_params['confidence']),
        'improvement': float(xy_params['improvement_percent'])
    },
    'depth_alignment': depth_alignment_params,
    'transform_3d': {
        'x_offset': int(offset_x),
        'y_shift': float(y_shift),
        'z_offset': int(offset_z),
        'x_rotation': float(best_x_angle),
        'z_rotation': float(best_z_angle)
    },
    'final_improvement': float(improvement)
}
np.save(data_dir / 'registration_3d_params.npy', registration_3d, allow_pickle=True)
print("✓ Saved: registration_3d_params.npy")

print("\n" + "="*70)
print("FINAL 3D REGISTRATION SUMMARY")
print("="*70)
print(f"\n📍 XZ Alignment:")
print(f"  X offset: {offset_x} px")
print(f"  Z offset: {offset_z} px")
print(f"\n📏 Y Alignment (center-based):")
print(f"  Y shift: {y_shift:+.2f} px")
print(f"\n🔄 Rotations:")
print(f"  X-axis: {best_x_angle}° (tilt in YZ)")
print(f"  Z-axis: {best_z_angle}° (rotation in YX)")
print(f"\n📊 Quality:")
print(f"  NCC: {ncc_before_rotation:.4f} → {ncc_final:.4f} (Δ {ncc_final-ncc_before_rotation:+.4f})")
print(f"  Improvement: {improvement:.2f}%")
print("\n✅ Phase 5 Complete!")
print("="*70)